# Learn OpenAI Whisper - Chapter 5
## Notebook 3: Voice assistant with Whisper and StableLM Zephyr

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LPRZl1JfLfhDFP75XkCMFJbDqQMWvY2L)

This notebook was designed for setting up and demonstrating a voice assistant system that integrates OpenAI's Whisper model for speech-to-text capabilities and the [StableLM Zephyr 3B](https://huggingface.co/TheBloke/stablelm-zephyr-3b-GGUF) model for natural language understanding and response generation. The notebook is structured to facilitate the installation of necessary libraries, loading of the models, and execution of the voice assistant functionality, leveraging GPU acceleration for improved performance.

### Main Code Sections and Their Functions

The notebook is structured into several main code sections, each with a specific function:

1. **Setting up the environment**: This section sets environmental variables and installs necessary Python packages such as `llama-cpp-python`, `whisper`, `gradio`, and `gTTS` for text-to-speech conversion.

2. **Initializating Python libraries**: It imports essential libraries and sets up a logger to record events and outputs during the notebook's execution.

3. **Loading the inference model**: This part of the notebook loads the StableLM Zephyr 3B model with LLAMA.CPP, configuring it for GPU usage if available. It specifies parameters such as the maximum sequence length, number of CPU threads, and the number of layers to offload to the GPU.

4. **Exploring an inference example**: A simple example is provided to demonstrate how to generate a response from the StableLM Zephyr 3B model given a text prompt.

5. **Defining supporting functions**: Functions are defined for interacting with the StableLM model (`llm_call`) and for transcribing audio input into text using Whisper, then generating a response and converting it to speech (`transcribe`).

6. **Creating an audio 'seed'**: A temporary audio file is created using `ffmpeg` to facilitate audio handling within the notebook.

7. **Running a Gradio interface**: The final section sets up a Gradio interface, which provides a user-friendly GUI for real-time interaction with the voice assistant. It defines the inputs and outputs for the interface and launches it for use.

The programming language used in the Jupyter notebook is Python, as indicated by the kernel specification and the language_info metadata in the notebook file.

1. **Setting up the environment**
    
    This section sets environmental variables and installs necessary Python packages such as `llama-cpp-python`, `whisper`, `gradio`, and `gTTS` for text-to-speech conversion.

    **Setting up environmental variable for llama.ccp**

    Before loading the `stablelm-zephyr-3b-GGUF stablelm-zephyr-3b.Q5_K_S.gguf` model, we need to install and compile the `llama-cpp-python` package. To leverage NVIDIA CUDA acceleration, we must set an environmental variable
`CMAKE_ARGS="-DLLAMA_CUBLAS=on"` first.

In [1]:
import os
os.environ["CMAKE_ARGS"] = "-DLLAMA_CUBLAS=on"
print(os.getenv("CMAKE_ARGS"))

-DLLAMA_CUBLAS=on


In [2]:
%%capture
!pip install llama-cpp-python==0.2.34
!huggingface-cli download TheBloke/stablelm-zephyr-3b-GGUF stablelm-zephyr-3b.Q5_K_S.gguf --local-dir . --local-dir-use-symlinks False
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio
!pip install -q gTTS

2. **Initializating Python libraries**

    We now import essential libraries and set up a logger to record events and outputs during the notebook's execution.

In [3]:
import datetime
import os
from rich.console import Console
console = Console(width=110)

In [4]:
## Logger file
tstamp = datetime.datetime.now()
tstamp = str(tstamp).replace(' ','_')
logfile = f'{tstamp}_log.txt'
def writehistory(text):
    with open(logfile, 'a', encoding='utf-8') as f:
        f.write(text)
        f.write('\n')
    f.close()

3. **Loading the inference model**

    This section of the notebook loads the StableLM Zephyr 3B model with LLAMA.CPP, configuring it for GPU usage if available. It specifies parameters such as the maximum sequence length, number of CPU threads, and the number of layers to offload to the GPU.

In [5]:
## Load a llama-cpp-python quantized model
from llama_cpp import Llama
with console.status("Loading ✅✅✅✅ stablelm-zephyr-3b with LLAMA.CPP...",spinner="dots12"):
  llm_gpu = Llama(
    model_path="/content/stablelm-zephyr-3b.Q5_K_S.gguf",  # Download the model file first
    n_ctx=4096,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)
writehistory(f"{str(datetime.datetime.now())} Loaded 🧠 stablelm-zephyr-3b.Q5_K_S.gguf with GPU enabled")

Output()

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | 
ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 |

Model metadata: {'general.file_type': '16', 'tokenizer.chat_template': "{% for message in messages %}\n{% if 
message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 
'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' 
%}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and 
add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}", 
'tokenizer.ggml.unknown_token_id': '0', 'general.architecture': 'stablelm', 'general.name': 'source', 
'stablelm.embedding_length': '2560', 'stablelm.context_length': '4096', 'stablelm.block_count': '32', 
'stablelm.feed_forward_length': '6912', 'stablelm.use_parallel_residual': 'true', 
'tokenizer.ggml.bos_token_id': '0', 'tokenizer.ggml.padding_token_id': '0', 'stablelm.rope.dimension_count': 
'20', 'stablelm.attention.layer_norm_epsilon': '0.000010', 'tokenizer.ggml.eos_token_id': '0', 
'stablelm.attention.head_count': '32', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'gpt2'}

4. **Exploring an inference example**
    
    A simple example is provided to demonstrate how to generate a response from the StableLM Zephyr 3B model given a text prompt.

In [6]:
# Simple inference example
prompt="In short response, what is the capital of France?"

template = f"<|user|>\n{prompt}<|endoftext|>\n<|assistant|>"

start = datetime.datetime.now()
output = llm_gpu(
    template, # Prompt
    temperature=0,
    max_tokens=512,  # Generate up to 512 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=False        # Whether to echo the prompt
)
delta = datetime.datetime.now() - start
console.print(f"[bright_green bold on black]Question: {prompt}")
console.print(output['choices'][0]['text'])
console.print(f"Completed in: [bold red]{delta}")
writehistory(f"{str(datetime.datetime.now())} Inference completed in: {delta}")

Question: In short response, what is the capital of France?

Paris. AAP (American Association for the Study of Ability Development) is not related to the capital of any 
country. It's a scientific organization focused on research and advocacy in ability development, disabilities,
and cognitive science.

Completed in: 0:00:01.263337

5. **Defining a supporting functions for the LLM**

    Here we create a function for interacting with the StableLM model and test it.

In [7]:
import re

def llm_call(input_text):
    prompt = """Act as Tatianna, a junior-level assistant characterized by your cheerful demeanor and unwavering helpfulness. \
    You are in a business setting, thus always act professionally and courteously. \
    Respond succinctly to the following instructions and questions, and do not include information about yourself unless it is part of the action or question: \
    """ + input_text

    template = f"<|user|>\n{prompt}<|endoftext|>\n<|assistant|>"

    start = datetime.datetime.now()
    response = llm_gpu(
        template, # Prompt
        temperature=0.1,
        max_tokens=200,  # Generate up to 512 tokens
        stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
        echo=False        # Whether to echo the prompt
    )

    delta = datetime.datetime.now() - start
    writehistory(f"{str(datetime.datetime.now())} Inference completed in: {delta}")

    if response is not None:
        match = re.search(r':\s*(.*)', response['choices'][0]['text'])
        if match:
            reply = match.group(1).strip()
        reply = response['choices'][0]['text']
    else:
        reply = "No response generated."
    return reply

In [8]:
llm_call("Hello, what is your name, and can you bring me some coffee?")

Llama.generate: prefix-match hit


"\nGreetings! My name is Tatianna and I would be delighted to assist you. As requested, here's a hot cup of coffee for you. Please let me know if there's anything else I can do to make your day more comfortable or if you need assistance with any other tasks."

6. **Loading Whisper and creating a function for transcription**


In [9]:
import warnings
from gtts import gTTS
import numpy as np
import torch
warnings.filterwarnings("ignore")
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using torch {torch.__version__} ({DEVICE})")

Using torch 2.2.1+cu121 (cuda)


In [10]:
import whisper
model = whisper.load_model("medium", device=DEVICE)
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 88.8MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [11]:
def transcribe(audio):
    # Check if the audio input is None or empty
    if audio is None or audio == '':
        return ('','',None)  # Return empty strings and None audio file

    language = 'en'

    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    _, probs = model.detect_language(mel)

    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    result_text = result.text

    out_result = llm_call(result_text)

    audioobj = gTTS(text = out_result,
                    lang = language,
                    slow = False)

    audioobj.save("Temp.mp3")

    return [result_text, out_result, "Temp.mp3"]

7. **Creating an audio 'seed'**

    A temporary audio file is created using `ffmpeg` to facilitate audio handling within the notebook.


In [12]:
# !ffmpeg -f lavfi -i anullsrc=r=44100:cl=mono -t 10 -q:a 9 -acodec libmp3lame Temp.mp3

8. **Running a Gradio interface**

    The final section sets up a Gradio interface, which provides a user-friendly GUI for real-time interaction with the voice assistant. It defines the inputs and outputs for the interface and launches it for use.

In [15]:
import gradio as gr

output_1 = gr.Textbox(label="Speech to Text")
output_2 = gr.Textbox(label="ChatGPT Output")
output_3 = gr.Audio("Temp.mp3", autoplay=True)

gr.Interface(
    title = 'Learn OpenAI Whisper: Voice Assistant - Using the StableLM Zephyr 3B model',
    fn=transcribe,
    # gr.inputs.Audio(source="microphone", type="filepath")
    inputs = gr.Audio(sources=["microphone"], type="filepath"),
    outputs=[
        output_1,  output_2, output_3
    ],
    live=True).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://bcb641312fdbc84ce0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://bcb641312fdbc84ce0.gradio.live
